In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install -q pyaudio

In [13]:
!pip install -q textblob

In [ ]:
pip install -q io wave

In [ ]:
!pip install webrtcvad

In [18]:
!pip install -q sounddevice

In [53]:
!pip install -q -U google-generativeai

In [19]:
!pip install -q edge-tts
!pip install -q asyncio

In [16]:
import json
import asyncio
import sounddevice as sd
import numpy as np
import edge_tts
import webrtcvad
import wave
import io
import google.generativeai as genai
import gradio as gr
import pyaudio
import wave

In [17]:
# Read the API key from the config.json file
with open('config.json', 'r') as file:
    config = json.load(file)
    api_key = config['api_key']

# Configure genai with the API key
genai.configure(api_key=api_key)

# Now you can use genai functions as needed
print("genai is configured with the API key.")


genai is configured with the API key.


In [ ]:
myfile = genai.upload_file("output_test.wav")
print(f"{myfile=}")

model = genai.GenerativeModel("gemini-1.5-flash")
result = model.generate_content([myfile, "You are an emotional assistant. You help people by listening to them and provide them with appropriate advice, but mainly you listen to them and lend them an ear. Give a suitable response based on the audio and maintain a conversational style. The response can be a follow up question, or a piece of advice, or anything that you deem as suitable as an emotional assistant"])

In [ ]:
print(result.text)

In [37]:
TEXT = result.text
VOICE = "en-US-JennyNeural"
OUTPUT_FILE = "test3.mp3"
WEBVTT_FILE = "test3.vtt"


async def amain() -> None:
    """Main function"""
    communicate = edge_tts.Communicate(TEXT, VOICE)
    submaker = edge_tts.SubMaker()
    with open(OUTPUT_FILE, "wb") as file:
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                file.write(chunk["data"])
            elif chunk["type"] == "WordBoundary":
                submaker.create_sub((chunk["offset"], chunk["duration"]), chunk["text"])

    with open(WEBVTT_FILE, "w", encoding="utf-8") as file:
        file.write(submaker.generate_subs())


if __name__ == "__main__":
    await(amain())

In [ ]:
import pyaudio
import webrtcvad
import collections
import time
import sys
import wave
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Audio recording parameters
RATE = 16000
CHANNELS = 1
FORMAT = pyaudio.paInt16
FRAME_DURATION = 30  # ms
FRAME_SIZE = int(RATE * FRAME_DURATION / 1000)  # Number of audio samples per frame

# Load configuration from config.json
def load_config():
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)
    return config

# Setup Google Cloud credentials from config.json
config = load_config()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config['api_key']

# Voice Activity Detector
vad = webrtcvad.Vad(2)  # Level 2 sensitivity

# Google Cloud Speech client
client = speech.SpeechClient()

def record_audio():
    """Records audio from the microphone in real-time."""
    audio_interface = pyaudio.PyAudio()

    # Set up audio stream
    stream = audio_interface.open(format=FORMAT,
                                  channels=CHANNELS,
                                  rate=RATE,
                                  input=True,
                                  frames_per_buffer=FRAME_SIZE)

    print("Recording... Press Ctrl+C to stop.")
    frames = collections.deque(maxlen=int(10 * 1000 / FRAME_DURATION))  # Store last 10 seconds of audio

    try:
        while True:
            frame = stream.read(FRAME_SIZE, exception_on_overflow=False)

            # Use VAD to check if the frame contains speech
            is_speech = vad.is_speech(frame, RATE)

            if is_speech:
                frames.append(frame)
            elif len(frames) > 0:
                # Process and transcribe audio when speech ends
                audio_data = b''.join(frames)
                transcribe_audio(audio_data)
                frames.clear()

    except KeyboardInterrupt:
        print("Stopping...")
        stream.stop_stream()
        stream.close()
        audio_interface.terminate()

def transcribe_audio(audio_data):
    """Transcribes the given audio data using Google Speech-to-Text."""
    audio = types.RecognitionAudio(content=audio_data)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US"
    )

    # Perform the transcription request
    response = client.recognize(config=config, audio=audio)

    # Print the transcription result
    for result in response.results:
        print("Transcription: {}".format(result.alternatives[0].transcript))

if __name__ == "__main__":
    record_audio()

In [ ]:
import whisper
import pyaudio
import numpy as np
import torch

# Load the Whisper model
model = whisper.load_model("base")

# Audio recording parameters
RATE = 16000  # Sample rate
CHANNELS = 1  # Number of input channels (mono)
FORMAT = pyaudio.paInt16  # 16-bit audio
CHUNK = 1024  # Buffer size

# Initialize PyAudio
audio_interface = pyaudio.PyAudio()

# Open the microphone stream
stream = audio_interface.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK
)

print("Recording... Press Ctrl+C to stop.")

# Helper function to convert audio buffer to numpy array
def buffer_to_numpy(audio_data):
    return np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

try:
    # Recording and transcription loop
    while True:
        # Read audio data from the stream
        audio_data = stream.read(CHUNK, exception_on_overflow=False)
        
        # Convert audio data to numpy array and perform transcription
        audio_array = buffer_to_numpy(audio_data)
        
        # Use Whisper's transcribe function on the audio chunk
        # To make transcription real-time, we use shorter audio chunks
        result = model.transcribe(audio_array, fp16=torch.cuda.is_available(), language="en", task="transcribe")
        
        # Print the transcription
        if "text" in result:
            print("Transcription:", result["text"])

except KeyboardInterrupt:
    # Cleanup when user interrupts the recording
    print("Stopping...")
    stream.stop_stream()
    stream.close()

In [ ]:
from playsound import playsound

audio_file = "output_test.wav"
playsound(audio_file)

print("Playback finished.")

In [3]:
!pip install -q simpleaudio

In [6]:
#Input speech from user
FRAMES_PER_BUFFER = 3200
CHANNELS = 1
FORMAT = pyaudio.paInt16
RATE = 16000
SILENCE_THRESHOLD = 100  # Amplitude threshold for silence detection
SILENCE_DURATION = 3  # Duration in seconds to consider as silence

p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=FRAMES_PER_BUFFER
)

print('start recording...')

frames = []
silent_chunks = 0
silence_limit = int(RATE / FRAMES_PER_BUFFER * SILENCE_DURATION)

while True:
    data = stream.read(FRAMES_PER_BUFFER)
    frames.append(data)

    # Convert audio data to numpy array to analyze sound level
    audio_data = np.frombuffer(data, dtype=np.int16)
    amplitude = np.abs(audio_data).mean()

    # Check if the amplitude is below the silence threshold
    if amplitude < SILENCE_THRESHOLD:
        silent_chunks += 1
    else:
        silent_chunks = 0

    # If silence has lasted for the specified duration, stop recording
    if silent_chunks > silence_limit:
        print("Silence detected. Stopping recording.")
        break

stream.stop_stream()
stream.close()
p.terminate()

# Save the recorded audio to a file
output_file = "output_test.wav"
with wave.open(output_file, 'wb') as obj:
    obj.setnchannels(CHANNELS)
    obj.setsampwidth(p.get_sample_size(FORMAT))
    obj.setframerate(RATE)
    obj.writeframes(b"".join(frames))

#Process the message using the model
myfile = genai.upload_file("output_test.wav")
print(f"{myfile=}")

model = genai.GenerativeModel("gemini-1.5-flash")
prompt = "You are an emotional assistant. You help people by listening to them and provide them with appropriate advice, but mainly you listen to them and lend them an ear. Give a suitable response based on the audio and maintain a conversational style. Acknowledge the positive things mentioned by them. Furthermore, the response can be a follow up question, or a piece of advice, or anything that you deem as suitable as an emotional assistant"
result = model.generate_content([myfile, ])

#Perform TTS conversion to play the output audio
TEXT = result.text
VOICE = "en-US-JennyNeural"
OUTPUT_FILE = "test3.mp3"
WEBVTT_FILE = "test3.vtt"


async def amain() -> None:
    """Main function"""
    communicate = edge_tts.Communicate(TEXT, VOICE)
    submaker = edge_tts.SubMaker()
    with open(OUTPUT_FILE, "wb") as file:
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                file.write(chunk["data"])
            elif chunk["type"] == "WordBoundary":
                submaker.create_sub((chunk["offset"], chunk["duration"]), chunk["text"])

    with open(WEBVTT_FILE, "w", encoding="utf-8") as file:
        file.write(submaker.generate_subs())


if __name__ == "__main__":
    await(amain())

start recording...
Silence detected. Stopping recording.
myfile=genai.File({
    'name': 'files/2j0nbtux8sp1',
    'display_name': 'output_test.wav',
    'mime_type': 'audio/wav',
    'sha256_hash': 'NDllMTZmNGYwNDBhOTc1NjE5YTJiMzVmNGI3MjNmMDMzMGFjNWU1ZjNkOGE3OTljNjM5NzEyODlmM2Q5OWU2MQ==',
    'size_bytes': '1369644',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/2j0nbtux8sp1',
    'create_time': '2024-10-19T14:16:22.068774Z',
    'expiration_time': '2024-10-21T14:16:21.994544844Z',
    'update_time': '2024-10-19T14:16:22.068774Z'})


In [15]:
import numpy as np
import wave
import pyaudio
import gradio as gr
import edge_tts
import asyncio
from textblob import TextBlob  # Import for sentiment analysis

# Audio settings
FRAMES_PER_BUFFER = 3200
CHANNELS = 1
FORMAT = pyaudio.paInt16
RATE = 16000
SILENCE_THRESHOLD = 100  # Amplitude threshold for silence detection
SILENCE_DURATION = 3  # Duration in seconds to consider as silence

# Initialize PyAudio
p = pyaudio.PyAudio()

# Initialize conversation context
conversation_history = []

# List of phrases that could indicate red flags
RED_FLAG_KEYWORDS = [
    "bad", "hate", "angry", "upset", "frustrated", "stress", "worried",
    "sick", "negative", "problem", "sad", "feel bad", "don't want",
    "no", "never", "can't", "avoid", "give up", "hopeless"
]

async def process_audio():
    # Start recording audio
    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=FRAMES_PER_BUFFER
    )

    print('start recording...')
    
    frames = []
    silent_chunks = 0
    silence_limit = int(RATE / FRAMES_PER_BUFFER * SILENCE_DURATION)

    while True:
        data = stream.read(FRAMES_PER_BUFFER)
        frames.append(data)

        # Convert audio data to numpy array to analyze sound level
        audio_data = np.frombuffer(data, dtype=np.int16)
        amplitude = np.abs(audio_data).mean()

        # Check if the amplitude is below the silence threshold
        if amplitude < SILENCE_THRESHOLD:
            silent_chunks += 1
        else:
            silent_chunks = 0

        # If silence has lasted for the specified duration, stop recording
        if silent_chunks > silence_limit:
            print("Silence detected. Stopping recording.")
            break

    stream.stop_stream()
    stream.close()

    # Save the recorded audio to a file
    output_file = "output_test.wav"
    with wave.open(output_file, 'wb') as obj:
        obj.setnchannels(CHANNELS)
        obj.setsampwidth(p.get_sample_size(FORMAT))
        obj.setframerate(RATE)
        obj.writeframes(b"".join(frames))
    
    return output_file

async def transcribe_audio(audio_file):
    """Transcribe the audio file to text using the generative model."""
    myfile = genai.upload_file(audio_file)
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    result = model.generate_content([myfile, "Transcribe what you hear in the audio file"])
    response_text = result.text
    return response_text

async def generate_response(audio_file):
    global conversation_history

    # Transcribe the audio file to text
    transcription = await transcribe_audio(audio_file)

    # Detect red flags in the transcription
    red_flag_detected, red_flag_message = detect_red_flags(transcription)

    # Process the audio file with your model
    myfile = genai.upload_file(audio_file)
    print(f"{myfile=}")

    model = genai.GenerativeModel("gemini-1.5-flash")
    
    # Build the context by combining previous messages
    context = "\n".join(conversation_history[-5:])  # Get the last 5 messages
    prompt = (f"You are an emotional assistant. You help people by listening to them and provide them "
              "with appropriate advice, but mainly you listen to them and lend them an ear. "
              f"Previous messages:\n{context}\n"
              "Give a suitable response based on the audio and maintain a conversational style. "
              "Acknowledge the positive things mentioned by them. "
              "Furthermore, the response can be a follow-up question, or a piece of advice, or anything "
              "that you deem as suitable as an emotional assistant.")

    result = model.generate_content([myfile, prompt])
    response_text = result.text
    conversation_history.append(response_text)  # Add response to history

    # Convert text response to speech and save as audio file
    response_audio_file = "response_audio.mp3"
    await text_to_speech(response_text, response_audio_file)

    return response_text, response_audio_file, red_flag_detected, red_flag_message

def detect_red_flags(transcription):
    """Detect red flags in the transcription based on sentiment analysis and keywords."""
    # Check for red flag keywords
    for keyword in RED_FLAG_KEYWORDS:
        if keyword in transcription.lower():
            return True, "Detected negative sentiment related to: " + keyword

    # Analyze sentiment
    sentiment = TextBlob(transcription).sentiment
    if sentiment.polarity < 0:  # Negative sentiment
        return True, "Detected negative sentiment."

    return False, ""

async def text_to_speech(text, output_file):
    """Convert text to speech using edge-tts and save as audio file."""
    communicate = edge_tts.Communicate(text, "en-US-JennyNeural")
    with open(output_file, "wb") as file:
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                file.write(chunk["data"])

async def talk():
    audio_file = await process_audio()  # Record audio
    response, audio_file_response, red_flag_detected, red_flag_message = await generate_response(audio_file)  # Generate response
    return response, audio_file_response, red_flag_detected, red_flag_message

def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("### Emotional Assistant")
        audio_button = gr.Button("Start Recording")
        output_text = gr.Textbox(label="Assistant Response", interactive=False)
        audio_output = gr.Audio(label="Response Audio", type="filepath")
        red_flag_output = gr.Textbox(label="Red Flag Detection", interactive=False)

        def start_recording():
            response, audio_file_response, red_flag_detected, red_flag_message = asyncio.run(talk())
            if red_flag_detected:
                red_flag_message = f"⚠️ Red Flag Detected: {red_flag_message}"
            else:
                red_flag_message = "No red flags detected."
            return response, audio_file_response, red_flag_message

        audio_button.click(fn=start_recording, outputs=[output_text, audio_output, red_flag_output])

    demo.launch()

if __name__ == "__main__":
    gradio_interface()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


start recording...
Silence detected. Stopping recording.


Traceback (most recent call last):
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\gradio\queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\gradio\blocks.py", line 2014, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\gradio\blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\an

start recording...
Silence detected. Stopping recording.


Traceback (most recent call last):
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\httplib2\__init__.py", line 1396, in _conn_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Python312\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Python312\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Python312\Lib\http\client.py", line 300, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\aswat\OneDrive\Documents\Voice\.venv\Lib\site-packages\gradio\queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^

In [19]:
def transcribe_audio(audio_file):
    myfile = genai.upload_file(audio_file)

    model = genai.GenerativeModel("gemini-1.5-flash")
    
    result = model.generate_content([myfile, "Transcribe what you hear in the audio file"])
    response_text = result.text
    print(response_text)

In [20]:
transcribe_audio('output_test.wav')

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond